In [1]:
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import configÇ

ModuleNotFoundError: No module named 'configÇ'

In [ ]:
from enum import Enum
class DFUData(Enum):
    L = 0 # Left Foot
    R = 1 # Right Foot
    L_LCA = 2  # Angiosom LCA Left Foot 
    R_LCA = 3  # Angiosom LCA Right Foot
    L_LPA = 4  # Angiosom LPA Left Foot 
    R_LPA = 5  # Angiosom LPA Right Foot 
    L_MCA = 6  # Angiosom MCA Left Foot 
    R_MCA = 7  # Angiosom MCA Right Foot 
    L_MPA = 8  # Angiosom MPA Left Foot 
    R_MPA = 9  # Angiosom MPA Right Foot 


class INAOEDataset():
    def __init__(self, root_path, dm_group = False):
        cg_path = os.path.join(root_path, 'Control Group')
        dm_path = os.path.join(root_path, 'DM Group')

        self.dm_group = dm_group

        self.cg_subjects = sorted(list(map(lambda x: os.path.join(cg_path, x), os.listdir(cg_path))))
        self.dm_subjects = sorted(list(map(lambda x: os.path.join(dm_path, x), os.listdir(dm_path))))

    def set_group(self, dm_group):
        self.dm_group = dm_group

    def __len__(self):
        if self.dm_group:
            return len(self.dm_subjects)
        
        return len(self.cg_subjects) 

    def get_subjects_names(self):
        dataset = self.dm_subjects if self.dm_group else self.cg_subjects
        return list(map(lambda x: os.path.basename(x), dataset))

    def get_subject(self, idx, data:DFUData):
        dataset = self.dm_subjects if self.dm_group else self.cg_subjects

        subject = os.path.basename(dataset[idx])
        path = os.path.join(dataset[idx], 'Angiosoms') if data.value > 1 else dataset[idx]
        return pd.read_csv(os.path.join(path, subject + '_' + data.name + '.csv'), header=None).astype('float32')

In [ ]:
dataset = INAOEDataset(config.INAOE_DATASET_DIR)

In [ ]:
from scipy.stats import skew, kurtosis

def get_metrics(dataset, data:DFUData):
    subjects = dataset.get_subjects_names()
    mean_data = []
    std_data = []
    max_data = []
    min_data = []
    skew_data = []
    kurtosis_data = []
    metrics = []

    fulldata = None
    for idx in range(len(dataset)):
        # data = dataset.get_subject(idx, data).to_numpy()
        subject_data = dataset.get_subject(idx, data).to_numpy()
        subject_data = subject_data[subject_data>0.5] # Remove background
        mean_data.append(subject_data.mean())
        std_data.append(subject_data.std())
        max_data.append(subject_data.max())
        min_data.append(subject_data.min())
        skew_data.append(skew(subject_data))
        kurtosis_data.append(kurtosis(subject_data))

    metrics = {'Subject': subjects,
        '{}_mean'.format(data.name): mean_data, 
        '{}_std'.format(data.name): std_data,
        '{}_min'.format(data.name): min_data,
        '{}_max'.format(data.name): max_data,
        '{}_skew'.format(data.name): skew_data,
        '{}_kurtosis'.format(data.name): kurtosis_data}

    return pd.DataFrame(metrics)

## Qualitative Test

In [ ]:
plt.figure(figsize=(16, 8))
for i in range(len(dataset)):
    plt.subplot(5, 15, i+1)
    data = dataset.get_subject(i, DFUData.L_LCA)
    plt.imshow(data.to_numpy())
    plt.xticks([])
    plt.yticks([])

plt.show()

# MPA

In [ ]:
l_mpa_metrics = get_metrics(dataset, DFUData.L_MPA)
r_mpa_metrics = get_metrics(dataset, DFUData.R_MPA)

l_mpa_metrics.to_csv('data/MPA/L_MPA_metrics.csv', index=False)
r_mpa_metrics.to_csv('data/MPA/R_MPA_metrics.csv', index=False)

# LCA

In [ ]:
l_lca_metrics = get_metrics(dataset, DFUData.L_LCA)
r_lca_metrics = get_metrics(dataset, DFUData.R_LCA)

l_lca_metrics.to_csv('data/LCA/L_LCA_metrics.csv', index=False)
r_lca_metrics.to_csv('data/LCA/R_LCA_metrics.csv', index=False)

# LPA

In [ ]:
l_lpa_metrics = get_metrics(dataset, DFUData.L_LPA)
r_lpa_metrics = get_metrics(dataset, DFUData.R_LPA)

l_lpa_metrics.to_csv('data/LPA/L_LPA_metrics.csv', index=False)
r_lpa_metrics.to_csv('data/LPA/R_LPA_metrics.csv', index=False)

# MCA

In [ ]:
l_mca_metrics = get_metrics(dataset, DFUData.L_MCA)
r_mca_metrics = get_metrics(dataset, DFUData.R_MCA)

l_mca_metrics.to_csv('data/MCA/L_MCA_metrics.csv', index=False)
r_mca_metrics.to_csv('data/MCA/R_MCA_metrics.csv', index=False)

# Full Foot

In [ ]:
l_metrics = get_metrics(dataset, DFUData.L)
r_metrics = get_metrics(dataset, DFUData.R)

l_metrics.to_csv('data/Full/L_metrics.csv', index=False)
r_metrics.to_csv('data/Full/R_metrics.csv', index=False)

# Angiosomes Metrics

In [ ]:
def angiosomes_metrics(dataset):
    angiosomes = [[DFUData.L_LCA, DFUData.R_LCA], [DFUData.L_LPA, DFUData.R_LPA], [DFUData.L_MCA, DFUData.R_MCA], [DFUData.L_MPA, DFUData.R_MPA]]
    
    angiosomes_names = ['LCA', 'LPA', 'MCA', 'MPA']
    subjects = dataset.get_subjects_names()

    for ang_idx, (left, right) in enumerate(angiosomes):
        mean_data = []
        std_data = []
        max_data = []
        min_data = []
        skew_data = []
        kurtosis_data = []
        current_angiosom = angiosomes_names[ang_idx]
        for idx in range(len(dataset)):
            l_data = dataset.get_subject(idx, left).to_numpy()
            r_data = dataset.get_subject(idx, right).to_numpy() 

            l_data = l_data[l_data>0.5] # Remove background
            r_data = r_data[r_data>0.5] # Remove background
            data = np.concatenate((l_data, r_data))

            mean_data.append(data.mean())
            std_data.append(data.std())
            max_data.append(data.max())
            min_data.append(data.min())
            skew_data.append(skew(data))
            kurtosis_data.append(kurtosis(data))

        metrics = {'Subjects': subjects,
            'mean': mean_data, 
            'std': std_data,
            'min': min_data,
            'max': max_data,
            'skew': skew_data,
            'kurtosis': kurtosis_data}

        df = pd.DataFrame(metrics)
        print('data/{}/{}_metrics.csv'.format(current_angiosom, current_angiosom))
        df.to_csv('data/{}/General_{}_metrics.csv'.format(current_angiosom, current_angiosom), index=False)

In [ ]:
angiosomes_metrics(dataset)

# General

In [ ]:
angiosomes = ['LCA', 'LPA', 'MCA', 'MPA']
mean_values = []
std_values = []
max_values = []
min_values = []

for angiosome in angiosomes:
    df = pd.read_csv('data/{}/General_{}_metrics.csv'.format(angiosome, angiosome))
    data = df['mean'].to_numpy()
    mean_values.append(data.mean())
    std_values.append(data.std())
    max_values.append(data.max())
    min_values.append(data.min())
    
metrics = {'angiosome': angiosomes,
        'mean': mean_values, 
        'std': std_values,
        'min': min_values,
        'max': max_values
    }

df = pd.DataFrame(metrics)
df.to_csv('data/General_metrics.csv', index=False)

In [7]:
df = pd.read_csv('data/LCA/L_LCA_metrics.csv') 
df.iloc[:, 1].mean()

26.56553733333334

In [8]:
df = pd.read_csv('data/LCA/R_LCA_metrics.csv') 
df.iloc[:, 1].mean()

26.623412666666663

In [14]:
df1 = pd.read_csv('data/MPA/L_MPA_metrics.csv') 
df1.iloc[:, 1].mean()

26.709402377777774

In [15]:
df2 = pd.read_csv('data/MPA/R_MPA_metrics.csv') 
df2.iloc[:, 1].mean()

26.754804000000004

In [19]:
pd.merge(df1, df2, on='Subject')

,Subject,L_MPA_mean,L_MPA_std,L_MPA_min,L_MPA_max,L_MPA_skew,L_MPA_kurtosis,R_MPA_mean,R_MPA_std,R_MPA_min,R_MPA_max,R_MPA_skew,R_MPA_kurtosis
0,CG001_M,25.358301,1.845612,21.924,28.470,-0.152682,-1.098475,25.844654,1.830448,22.062,28.834,-0.097359,-1.070884
1,CG002_M,28.391384,1.411418,24.596,30.543,-0.359416,-0.628457,28.403757,1.381560,24.947,30.584,-0.420391,-0.586003
2,CG003_M,29.053340,1.309371,23.952,32.286,0.052050,-0.226531,29.743803,0.951396,27.849,32.452,0.687960,0.322575
3,CG004_F,26.839620,2.046408,22.283,31.195,-0.074981,-0.579650,27.565521,1.854106,24.153,31.661,0.119196,-0.649258
4,CG005_F,26.886417,1.851659,23.200,29.897,-0.136277,-1.100463,26.251804,1.851993,22.899,29.289,-0.070472,-1.117860
5,CG006_F,26.566670,1.661104,23.200,30.512,0.560160,-0.170446,26.266872,1.383340,23.424,30.108,1.191617,1.048334
6,CG007_F,26.943684,1.901957,22.679,30.010,-0.400926,-0.656595,26.239662,2.066529,21.363,29.800,-0.415835,-0.800701
7,CG008_F,24.377213,1.431968,21.000,26.855,-0.680097,-0.285282,25.084543,1.549532,20.637,27.697,-0.817043,0.352799
8,CG009_M,25.709661,2.687215,21.313,29.820,-0.049755,-1.347951,25.512075,2.548631,20.597,29.882,-0.080336,-1.130949
9,CG010_M,26.671373,1.348771,23.816,29.026,0.025485,-0.931324,26.852705,1.349465,24.272,29.547,0.405892,-0.916108


In [18]:
df1

,Subject,L_MPA_mean,L_MPA_std,L_MPA_min,L_MPA_max,L_MPA_skew,L_MPA_kurtosis
0,CG001_M,25.358301,1.845612,21.924,28.470,-0.152682,-1.098475
1,CG002_M,28.391384,1.411418,24.596,30.543,-0.359416,-0.628457
2,CG003_M,29.053340,1.309371,23.952,32.286,0.052050,-0.226531
3,CG004_F,26.839620,2.046408,22.283,31.195,-0.074981,-0.579650
4,CG005_F,26.886417,1.851659,23.200,29.897,-0.136277,-1.100463
5,CG006_F,26.566670,1.661104,23.200,30.512,0.560160,-0.170446
6,CG007_F,26.943684,1.901957,22.679,30.010,-0.400926,-0.656595
7,CG008_F,24.377213,1.431968,21.000,26.855,-0.680097,-0.285282
8,CG009_M,25.709661,2.687215,21.313,29.820,-0.049755,-1.347951
9,CG010_M,26.671373,1.348771,23.816,29.026,0.025485,-0.931324
